In [1]:
##This reference design will help you walk through the rules of the contest.
##Any change in DACSDC.py will make your design fail in evaluation. 
##Only "resize" is allowed in PS side. If any inference is executed in PS, it would be regarded as fail.
##Time for a batch processing should start from 'PS reading images' and end after 'PS receving all results'. Please check the reference for details.
##Batch size is 500 by defalut.
##Please all "DACSDC.write" to write your time performance.
##Please all "storeResultsToXML.write" to write your time performance.
##It is your choice how to record the inferenced result. However, it must be readable, and you must convert it to XML files. An example is provided.
##Please pack your code into teamname.py. Your design should be able to process all evaluation images with one click.
##xinyizhang@pitt.edu, University of Pittsburgh
    

In [14]:
###start initializing###
import sys
import numpy as np 
import os
import time
from PIL import Image
import matplotlib.pylab as plt
import cv2
from datetime import datetime
from pynq import Xlnk
from pynq import Overlay
import shutil 
import DACSDC
##Please enter you team name here; set the sleep time for function send, it is 0 by default
teamname = 'OSSDC'
interval_time= 0
batchsize = 500
##create default work dir
[DAC,imgdir,overlaydir,result,timedir,coordir,xml,mycoord,myxml]=DACSDC.startup(teamname)
##mycoord: please keep your raw data here.
##myxml:  please keep your xml files here.

###end initializing###


In [15]:
##Reset the global count
##Process images in batch

sendlist = DACSDC.getpathbatch()
totalimg = len(DACSDC.getnames())
print (totalimg)

6


In [16]:
DACSDC.reset()

In [17]:
##this is not a real design for image dectection, this is not a real design for image dectection   
##start your design
overlay = Overlay('/home/xilinx/jupyter_notebooks/DAC/overlay/Su.bit')
dma = overlay.axi_dma_0
f = []
xlnk = Xlnk()
in_buffer = xlnk.cma_array(shape=(4,), dtype=np.uint32)
out_buffer = xlnk.cma_array(shape=(4,), dtype=np.uint32)
tbatch = 0
##Data from PS->PL->PS by DMA. Using "Interrupt" is suggested by Xilinx engineers. 
##More details can be found in the reference BD files.
def push (indata):
    for k in range (4):
        in_buffer[k] = indata[k]
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer) 
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    return out_buffer

##You must iterate all batches. Here, t1 just grabs 4 int from a batch and then loops them back.
##It is your turn how to record the output raw data. You can put batches results in separate files or a single file. 
##You should measure the time for processing all images.
##Timers should start from 'PS reading images' and end after 'PS receving results'
def t1(): 
    for i in range (int(totalimg/batchsize)):
        global tbatch
        start = time.time()
        print ("this is t1's number %s batch"%(i))
        a  = DACSDC.send(interval_time, sendlist)
        d= a[0]
        d = cv2.imread(d)
        b,g,r = cv2.split(d)
        e = b [0]
        push (e)
        
        end = time.time()
        t= end -start
        print (t)
        tbatch = tbatch + t 
        print (tbatch)
        
        out = str (out_buffer)
        print (out)
        f.append(out)
    return



t1()
##Please call "DACSDC.write" to record your time performance. 
DACSDC.write(tbatch, totalimg,teamname)
##Please write your coordinates to "'DAC/result/coordinate/teamname/...'"
##Please put all your intermediate result here, it is your choice how to store them.
fcoord = open(mycoord + '/' + teamname + '.txt', 'w+')
for element in f: 
    fcoord.write(element)  
    fcoord.write('\n')  
fcoord.close()
    
xlnk = Xlnk()
xlnk.xlnk_reset()



this is t1's number 0 batch
0.031674861907958984
0.031674861907958984
[255 255 255 255]
this is t1's number 1 batch
0.03101372718811035
0.06268858909606934
[255 255 255 255]
this is t1's number 2 batch
0.028666973114013672
0.09135556221008301
[255 255 255 255]


In [ ]:
##call this function to write to XML
DACSDC.storeResultsToXML(resultRectangle, allImageName, myxml):